In [1]:
!pip install -q sentence-transformers
!pip install -q langchain langchain_community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00


In [3]:
!npm install -q localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [4]:
!pip install python-dotenv

In [5]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 77.4 MB/s eta 0:00:00


In [6]:
import faiss
from langchain_community.vectorstores import FAISS

In [7]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.1 MB/s eta 0:00:00


In [10]:
%%writefile .env
HUGGINGFACE_API_KEY="Your API Key"
HUGGINGFACEHUB_API_KEY="Your API Key"
HF_TOKEN="Your API Key"

Writing .env


In [8]:
# from huggingface_hub import login
# login("Your API Key")

In [11]:
import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

import torch
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint

from dotenv import load_dotenv
load_dotenv()

True

In [16]:
%%writefile chatdocument.py

import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

import torch
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEndpoint

import faiss
import tempfile
import os
import time

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader

from dotenv import load_dotenv

load_dotenv()

#Streamlit setting
st.set_page_config(page_title="Chat with Documents 📚", page_icon="📚")
st.title("Chat with Documents 📚")


def model_hf_hub(model="microsoft/Phi-3.5-mini-instruct", temperature=0.1):
    llm = HuggingFaceEndpoint(
        repo_id = model,
        temperature = temperature,
        max_new_tokens = 512,
        return_full_text = False,
    )

    return llm




#Indexing and retrieval
def config_retrieval(uploads):
    #Load
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploads:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyPDFLoader(temp_filepath)
        docs.extend(loader.load())

    #Split
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200
    )
    splits = text_splitter.split_documents(docs)

    #Embedding
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

    #Store
    vectorstore = FAISS.from_documents(splits, embeddings)
    vectorstore.save_local('vectorstore/db_faiss')

    #Retrieve
    retriever = vectorstore.as_retriever(
        search_type = "mmr",
        search_kwargs = {
            'k': 3,
            'fetch_k': 4
        }
        ) #Maximum Marginal Relevant (mmr)
    return retriever


def config_rag_chain(retriever):
    llm = model_hf_hub()
    token_s, token_e = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>", "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

    #query -> retriever
    #(query, chat_history) -> LLM -> reformulated query -> retriever
    #Contextualization Prompt
    context_q_system_prompt = "Given the following chat history and the follow-up question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
    context_q_system_prompt = token_s + context_q_system_prompt
    context_q_user_prompt = "Question: {input}" + token_e
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ('system', context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ('human', context_q_user_prompt)
        ]
    )

    #Chain for contextualization
    history_aware_retriever = create_history_aware_retriever(llm = llm, retriever = retriever, prompt = context_q_prompt)

    #Q&A Prompt
    qa_prompt_template = """You are a helpful virtual assistant answering general questions.
    Use the following bits of retrieved context to answer the question.
    If you don't know the answer, just say you don't know. keep your answer concise.
    Answer in English. \n\n
    Question: {input} \n
    Context: {context}"""

    qa_prompt = PromptTemplate.from_template(token_s + qa_prompt_template + token_e)

    #Configure LLM and Chain for Q&A
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

    return rag_chain


#Create side panel in interface
uploads = st.sidebar.file_uploader(
    label = "Upload files",
    type = ["pdf"],
    accept_multiple_files = True
)

if not uploads:
    st.info("Please send some files to continue")
    st.stop()

if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        AIMessage(content = "Hi, I'm your virtual assistant! How can I help you?")
    ]

if "docs_list" not in st.session_state:
    st.session_state.docs_list = None

if "retriever" not in st.session_state:
    st.session_state.retriever = None


for message in st.session_state.chat_history:
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.write(message.content)
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.write(message.content)

user_query = st.chat_input("Enter your message here...")

if user_query is not None and user_query != "" and uploads is not None:
    st.session_state.chat_history.append(HumanMessage(content = user_query))
    with st.chat_message("Human"):
        st.markdown(user_query)
    with st.chat_message("AI"):
        if st.session_state.docs_list != uploads:
            st.session_state.docs_list = uploads
            st.session_state.retriever = config_retrieval(uploads)
        rag_chain = config_rag_chain(st.session_state.retriever)
        result = rag_chain.invoke({"input": user_query, "chat_history": st.session_state.chat_history})

        resp = result['answer']
        st.write(resp)


        #Show the source
        sources = result['context']
        for idx, doc in enumerate(sources):
            source = doc.metadata['source']
            file = os.path.basename(source)
            page = doc.metadata.get('page', 'Page not specified')
            ref = f":link: Source {idx}: *{file} - p. {page}*"
            with st.popover(ref):
                st.caption(doc.page_content)
    st.session_state.chat_history.append(AIMessage(content = resp))



Overwriting chatdocument.py


In [17]:
!streamlit run chatdocument.py &>/content/logs.txt &

In [18]:
!wget -q -O - ipv4.icanhazip.com

34.82.200.229


In [19]:
!npx localtunnel --port 8501

⠙your url is: https://quiet-lines-type.loca.lt
^C
